In [ ]:
import os
import cv2
import numpy as np
import tifffile as tiff
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import  Dropout, Input, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Variables
VAL_SPLIT = 0.2
BATCH_SIZE = 32
READ_SIZE = (128, 128)
MODEL_INPSHAPE = (128, 128, 3)
LABEL_CLASSES = ['no-damage', 'minor-damage', 'major-damage', 'destroyed', 'un-classfied']
NUM_LABELS = len(LABEL_CLASSES)
TRAIN_DIR = ""
TEST_DIR = ""

In [ ]:
#Read Images
train_data = ImageDataGenerator(validation_split=VAL_SPLIT)
test_data = ImageDataGenerator()

train_gen = image_data.flow_from_directory(TRAIN_DIR, target_size=READ_SIZE, class_mode=LABEL_CLASSES, 
                                           batch_size=BATCH_SIZE)
val_gen = image_data.flow_from_directory(TRAIN_DIR, target_size=READ_SIZE, class_mode=LABEL_CLASSES, 
                                           batch_size=BATCH_SIZE)
test_gen = image_data.flow_from_directory(TEST_DIR, target_size=READ_SIZE, class_mode=LABEL_CLASSES, 
                                          batch_size=BATCH_SIZE)


In [ ]:
#Visualize the images
tiff.imshow(test_gen.next()[0])

In [ ]:
#vgg model with pretrained weights
def create_vgg16_model():
    vgg_model = tensorflow.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=MODEL_INPSHAPE))

    # Add Unfreezing last 4 layers for transfer learning
    for layer in vgg_model.layers[:-4]:
        layer.trainable = False

    model = Sequential()
    model.add(vgg_model)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_LABELS, activation='softmax'))
    
    return model

In [ ]:
model = create_vgg16_model
model.summary()  

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [ ]:
#Checkpoint to save model
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='cnnvgg16.hdf5',
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_freq='epoch',
    save_best_only=True)

In [ ]:
history = model.fit(train_gen, steps_per_epoch = train_gen.samples // BATCH_SIZE, 
                    validation_data = val_gen, validation_steps = val_gen.samples // BATCH_SIZE, 
                    epochs=20, callbacks=[model_checkpoint_callback])

In [ ]:
#Plot Graphs
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter( y=history.history['val_loss'], name="val_loss"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter( y=history.history['loss'], name="loss"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter( y=history.history['val_accuracy'], name="val accuracy"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter( y=history.history['accuracy'], name="accuracy"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Loss/Accuracy of Sequential CNN Model"
)

# Set x-axis title
fig.update_xaxes(title_text="Epoch")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> Loss", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> Accuracy", secondary_y=True)

fig.show()


In [ ]:
#Test Data
test_imgs, test_labels = next(test_gen)
predictions = model.predict(test_imgs)